# View Color light-Curves and Histograms for repeatability

- author Sylvie Dagoret-Campagne
- creation date 2024-09-07
- last update 2024-09-07

In [1]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

In [3]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

import scipy
from scipy.optimize import curve_fit

In [12]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
theta_sep = 300.
time_sep = 1800.

path_colorlc = f"data_coloredlightcurvesG_R_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}"

summary_file = f"photompairsummaryG_R_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}.csv"
fntag = f"^color_lightcurveG_R_(.*)_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}.csv$"

summary_fullfilename= os.path.join(path_colorlc,summary_file) 

dfc = pd.read_csv(summary_fullfilename,index_col=0)

In [9]:
dfc

,idx_pair,idx_obj1,idx_obj2,sep,refcol_psf_md,n_refcol_psf,refcol_ap_md,n_refcol_ap,muclip,sigmaclip,nclip,magmax
0,0.0,141.0,907.0,7.0,0.245792,54.0,0.241321,54.0,0.243219,0.021885,54.0,16.324735
1,1.0,654.0,669.0,10.0,0.201248,14.0,0.197477,14.0,0.197428,0.009260,14.0,15.166812
2,2.0,111.0,262.0,16.0,0.390104,102.0,0.396171,102.0,0.396514,0.015744,102.0,15.853043
3,3.0,813.0,950.0,16.0,0.516337,48.0,0.519901,48.0,0.520206,0.026406,48.0,16.395164
4,4.0,64.0,262.0,18.0,0.334823,96.0,0.340353,96.0,0.340714,0.014137,96.0,15.777518
...,...,...,...,...,...,...,...,...,...,...,...,...
973,997.0,141.0,654.0,299.0,0.415387,11.0,0.400365,11.0,0.396339,0.016838,11.0,15.417780
974,998.0,392.0,654.0,299.0,0.175354,8.0,0.140779,8.0,0.140760,0.005477,8.0,14.342296
975,999.0,369.0,721.0,299.0,0.093145,2.0,0.083797,2.0,0.083797,0.036565,2.0,15.652288
976,1000.0,201.0,392.0,299.0,0.136615,36.0,0.139459,36.0,0.141339,0.014815,36.0,15.738955


In [10]:
len(dfc.idx_pair.unique())

978

In [11]:
list_of_files = os.listdir(path_colorlc) 

In [13]:
tag

'^color_lightcurveG_R_(.*)_thmax_300_tmax_1800.csv$'

In [15]:
all_pairnum = []
for idx,file in enumerate(list_of_files):
    coll = re.findall(tag,file)
    if len(coll)>0:
        all_pairnum.append(int(coll[0]))
        
all_pairnum = sorted(all_pairnum)  

In [17]:
print(all_pairnum) 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [23]:
all_df = []
for idx,idx_pair in enumerate(all_pairnum):
    idx_pair_str = str(idx_pair).zfill(4)
    filename =  f"color_lightcurveG_R_{idx_pair_str}_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}.csv"
    fullfilename = os.path.join(path_colorlc,filename)
    df = pd.read_csv(fullfilename,index_col=0)
    all_df.append(df)
    

In [25]:
df = all_df[0]

### Definitions

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            
            if FLAG_AP_FLUX_CAL:
                apMag11 =  row_pair_b1["apFlux_35_0_calMag_x"]
            else:
                apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
 
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]


            if FLAG_AP_FLUX_CAL:
                apMag12 =  row_pair_b1["apFlux_35_0_calMag_y"]
  
            else:
                apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
 
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]

            if FLAG_AP_FLUX_CAL:
                apMag21 =  row_pair_b2["apFlux_35_0_calMag_x"]
            else:
                apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])

            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]

            
            if FLAG_AP_FLUX_CAL:
                apMag22 =  row_pair_b2["apFlux_35_0_calMag_y"]
            else:
                apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])

        

In [27]:
df.head()

,band1,exposure1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,band2,exposure2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,airmassb1,airmassb2,x11,y11,x12,y12,x21,y21,x22,y22,deltapsfmag_s12b1,deltapsfmagErr_s12b1,deltaapmag_s12b1,deltaapmagErr_s12b1,deltapsfmag_s12b2,deltapsfmagErr_s12b2,deltaapmag_s12b2,deltaapmagErr_s12b2,psfcol12_s1,psfcol12Err_s1,apcol12_s1,apcol12Err_s1,psfcol12_s2,psfcol12Err_s2,apcol12_s2,apcol12Err_s2,psfcol12_s12,psfcol12Err_s12,apcol12_s12,apcol12Err_s12,psfmagmax_s12b1,psfmagmax_s12b2,apmagmax_s12b1,apmagmax_s12b2,index,DATE-OBS,id,BITPIX,NAXIS,NAXIS1,NAXIS2,EXTEND,TARGET,REDSHIFT,GRATING,ROTANGLE,D2CCD,LSHIFT,PARANGLE,TARGETX,TARGETY,LBDA_REF,PSF_REG,TRACE_R,MEANFWHM,PIXSHIFT,CHI2_FIT,A2_FIT,AM_FIT,VERSION,CCD_REBIN,CONFIG,EXPTIME,AIRMASS,DEC,HA,OUTTEMP,OUTPRESS,OUTHUM,FILTER,CAM_ROT,S_X0,S_Y0,S_XMIN,S_XMAX,S_YMIN,S_YMAX,S_NX,S_NY,S_DEG,S_SAT,S_ORDER,A1_x,A1_err_x,A2_x,A2_err_x,A3,A3_err,VAOD_x,VAOD_err_x,angstrom_exp_x,angstrom_exp_err_x,ozone [db]_x,ozone [db]_err_x,PWV [mm]_x,PWV [mm]_err_x,B_x,B_err_x,A_star,A_star_err,D_CCD [mm]_x,D_CCD [mm]_err_x,shift_x [pix],shift_x [pix]_err,shift_y [pix],shift_y [pix]_err,angle [deg],angle [deg]_err,x_c_0_1,x_c_0_1_err,x_c_1_1,x_c_1_1_err,x_c_2_1,x_c_2_1_err,y_c_0_1,y_c_0_1_err,y_c_1_1,y_c_1_1_err,y_c_2_1,y_c_2_1_err,gamma_0_1,gamma_0_1_err,gamma_1_1,gamma_1_1_err,gamma_2_1,gamma_2_1_err,alpha_0_1,alpha_0_1_err,alpha_1_1,alpha_1_1_err,alpha_2_1,alpha_2_1_err,saturation_0_1,saturation_0_1_err,x_c_0_2,x_c_0_2_err,x_c_1_2,x_c_1_2_err,x_c_2_2,x_c_2_2_err,y_c_0_2,y_c_0_2_err,y_c_1_2,y_c_1_2_err,y_c_2_2,y_c_2_2_err,gamma_0_2,gamma_0_2_err,gamma_1_2,gamma_1_2_err,gamma_2_2,gamma_2_2_err,alpha_0_2,alpha_0_2_err,alpha_1_2,alpha_1_2_err,alpha_2_2,alpha_2_2_err,saturation_0_2,saturation_0_2_err,x_c_0_3,x_c_0_3_err,x_c_1_3,x_c_1_3_err,x_c_2_3,x_c_2_3_err,y_c_0_3,y_c_0_3_err,y_c_1_3,y_c_1_3_err,y_c_2_3,y_c_2_3_err,gamma_0_3,gamma_0_3_err,gamma_1_3,gamma_1_3_err,gamma_2_3,gamma_2_3_err,alpha_0_3,alpha_0_3_err,alpha_1_3,alpha_1_3_err,alpha_2_3,alpha_2_3_err,saturation_0_3,saturation_0_3_err,A1_y,A1_err_y,A2_y,A2_err_y,VAOD_y,VAOD_err_y,angstrom_exp_y,angstrom_exp_err_y,ozone [db]_y,ozone [db]_err_y,PWV [mm]_y,PWV [mm]_err_y,reso [nm],reso [nm]_err,D_CCD [mm]_y,D_CCD [mm]_err_y,alpha_pix [pix],alpha_pix [pix]_err,B_y,B_err_y,Time,dt_abs_specphot,td_rel_specphot
0,g,2023112700345,2023-11-28 07:24:53.839493888,15.394361,16.341429,0.005133,0.008551,15.384073,16.361838,0.006894,0.016704,r,2023112700346,2023-11-28 07:25:40.615503872,14.642168,15.317250,0.003681,0.005035,14.629194,15.324140,0.004297,0.007885,46.776009,1.030945,1.030285,2001.208579,733.366565,2065.686134,785.999627,2002.672080,733.573780,2067.138841,786.155962,-0.947068,0.009974,-0.977765,0.018071,-0.675082,0.006237,-0.694946,0.008980,0.752193,0.006317,0.754879,0.008124,1.024179,0.009923,1.037698,0.018472,0.271986,0.011763,0.282819,0.020179,16.341429,15.317250,16.361838,15.324140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,g,2023112700353,2023-11-28 07:32:33.813998080,15.393639,16.321255,0.004822,0.007620,15.396599,16.330897,0.007165,0.016071,r,2023112700358,2023-11-28 07:37:10.383496960,14.669641,15.352447,0.003843,0.005174,14.661529,15.356039,0.004323,0.007601,276.569498,1.024954

In [ ]:
len(dfc[dfc.obj_index == all_objnum[0]] )

In [ ]:
residuals = {}
avgpsfmag = {}
residuals["g"] = [] 
residuals["r"] = [] 
residuals["i"] = [] 
residuals["z"] = [] 
residuals["y"] = [] 
avgpsfmag["g"] = []
avgpsfmag["r"] = []
avgpsfmag["i"] = []
avgpsfmag["z"] = []
avgpsfmag["y"] = []
FLAG_PSF = False
FLAG_CALIB = False

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 
colors = ["g","r","orange","grey","k"]

# loop on individual objects
for idx,df in enumerate(all_df):
    obj_num = df.obj_index.unique()[0]
    
    df_filt = {}
    
    # loop on filters
    
    for idx_b,b in enumerate(["g","r","i","z","y"]):
        df_filt[b] = df[df.band == b]
        
        
    fig,axs = plt.subplots(5,1,figsize=(12,8),sharex=True)
    
    axes_flat = axs.flatten()
    for idx_b,b in enumerate(["g","r","i","z","y"]):
        X = pd.to_datetime(df_filt[b]['obsStart'].values)
        
        
        if FLAG_PSF:    
            Y = df_filt[b]['psfMag'].values
            Yerr = df_filt[b]['psfMagErr'].values        
        else:
            if FLAG_CALIB:
                Y = df_filt[b]['apFlux_35_0_calMag'].values
                Yerr = df_filt[b]['apFlux_35_0_calMagErr'].values
            else:
                #Y = df_filt[b]['apFlux_35_0_instMag'].values
                #Yerr = df_filt[b]['apFlux_35_0_instMagErr'].values
                Y = df_filt[b]["apFlux_35_0_instMag"].values
                Yerr = df_filt[b]["apFlux_35_0_instMagErr"].values
                
                
                 
        
        N = len(Y)
        clippedMeanMagArray = scipy.stats.sigmaclip(Y, low=3.0, high=3.0).clipped
        clippedMeanMag = np.mean(clippedMeanMagArray)
        clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))
        clippeErrorMagArray = scipy.stats.sigmaclip(Y-clippedMeanMag,low=3.0, high=3.0).clipped
        clippeErrorMagSigma = (np.sqrt(np.sum(clippeErrorMagArray**2))/np.sqrt(len(clippeErrorMagArray)))*1000.

        if b in ["g","r","z","y"]:
                avgpsfmag[b].append(clippedMeanMag)
                residuals[b].append(clippeErrorMagArray)
              
        
        label = f" band {b} : mag = {clippedMeanMag:.1f} mag , rms = {clippeErrorMagSigma:.0f} mmag, N= {N}"
        axes_flat[idx_b].errorbar(X,(Y-clippedMeanMag)*1000,yerr=Yerr*1000,fmt='.',lw=0.5,c=colors[idx_b],ecolor=colors[idx_b]) 
        axes_flat[idx_b].scatter(X,Y-clippedMeanMag,marker='o',s=10,c=colors[idx_b],label=label) 
        axes_flat[idx_b].xaxis.set_major_locator(month_locator) # Locator for major axis only.
        axes_flat[idx_b].xaxis.set_minor_locator(day_locator)
        axes_flat[idx_b].xaxis.set_major_formatter(year_month_formatter)
        axes_flat[idx_b].set_xticks(axes_flat[idx_b].get_xticks(), axes_flat[idx_b].get_xticklabels(), rotation=45, ha='right')
        axes_flat[idx_b].grid()
        if idx_b == 0:
            if FLAG_PSF:  
                axes_flat[idx_b].set_title(f"psf Magnitude for object {obj_num}")
            else:
                if FLAG_CALIB:
                    axes_flat[idx_b].set_title(f"Aperture Magnitude (calib) for object {obj_num}")
                else:
                    axes_flat[idx_b].set_title(f"Aperture Magnitude (instrum) for object {obj_num}")
                    
                
        elif idx_b == 4:
            axes_flat[idx_b].set_xlabel("date obs") 
        axes_flat[idx_b].set_ylabel("dev (mmag)") 
        axes_flat[idx_b].legend(loc="upper left")
        axes_flat[idx_b].set_ylim(-50,50)
    plt.tight_layout()
    plt.show()
    
  
    
    

## Histograms

In [ ]:
band = "g"

In [ ]:
color = ["orange","k","g","b","r"]
thresholds = np.array([13.,14.,15.,16.,17.])
histoList1 = []
histoList2 = []
histoList3 = []
histoList4 = []
histoList5 = []
histoLists = [ histoList1, histoList2, histoList3, histoList4,histoList5]
Nthr = len(thresholds)

In [ ]:
the_list_of_LC_average = avgpsfmag[band]
the_list_of_LC_residuals = residuals[band]
NLC = len(the_list_of_LC_average)

## Loop on threshold
for ith,th in enumerate(thresholds):
    for iobs in range(NLC):
        mag = the_list_of_LC_average[iobs]
        res = the_list_of_LC_residuals[iobs] 
        if mag < th:
            histoLists[ith].append(res)

In [ ]:
for ith,th in enumerate(thresholds):
    print(ith,th,len(histoLists[ith]))
    histoLists[ith] = np.hstack(histoLists[ith])
    print(ith,th,len(histoLists[ith]))
    histoLists[ith]*=1000.  # convert in mmag


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5))
for ith,th in enumerate(thresholds):
    data = histoLists[4-ith]
    sigma = np.std(data)
    m = thresholds[4-ith]
    label = f" m < {m:.0f} mag\n$\sigma$ = {sigma:.1f} mmag"
    ax.hist(data,bins=50,range=(-100,100),facecolor=color[4-ith],alpha=0.9,label=label)
ax.legend(loc="upper left")
if FLAG_PSF:
    ax.set_title(f"Light curve residuals in band {band} (PSF Flux)")
else:
    if FLAG_CALIB:
        ax.set_title(f"Light curve residuals in band {band} (calib Aperture Flux)")
    else:
        ax.set_title(f"Light curve residuals in band {band} (instrum Aperture Flux)")
        
ax.set_xlabel("residuals (mmag)")

## error vs mag

In [ ]:
df_allsources.head() 

In [ ]:
df_allsources["psfMagErr_mmag"] =df_allsources["psfMagErr"]*1000.
df_allsources["apFlux_35_0_calMagErr_mmag"] =df_allsources["apFlux_35_0_calMagErr"]*1000.
df_allsources["apFlux_35_0_instMagErr_mmag"] =df_allsources["apFlux_35_0_instMagErr"]*1000.

df_allsources_g = df_allsources[df_allsources.band == "g" ]
df_allsources_r = df_allsources[df_allsources.band == "r" ]
df_allsources_i = df_allsources[df_allsources.band == "i" ]
df_allsources_z = df_allsources[df_allsources.band == "z" ]
df_allsources_y = df_allsources[df_allsources.band == "y" ]

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12,8))

ax1,ax2,ax3,ax4 = axs.flatten()

df_allsources_g.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="k",alpha=0.1)
ax1.set_title("PSF magnitude (calibrated)")

df_allsources_g.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="k",alpha=0.1)
ax2.set_title("Aperture magnitude (calibrated with localZeroPoint)")

df_allsources_g.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="k",alpha=0.1)
ax3.set_title("Aperture instrumental magnitude (scaled with CCD ZeroPoint)")

ax4.set_axis_off()


ax1.legend()
ax1.set_xlim(11.,17)
ax1.set_ylim(0.,20.)
ax2.legend()
ax2.set_xlim(11.,17)
ax2.set_ylim(0.,20.)
ax3.legend()
ax3.set_xlim(11.,17)
ax3.set_ylim(0.,20.)


# generate legend
import matplotlib.patches as mpatches
green_patch = mpatches.Patch(color='green', label='g band',alpha=0.5)
red_patch = mpatches.Patch(color='red', label='r band',alpha=0.5)
orange_patch = mpatches.Patch(color='orange', label='i band',alpha=0.5)
grey_patch = mpatches.Patch(color='grey', label='z band',alpha=0.5)
black_patch = mpatches.Patch(color='k', label='y band',alpha=0.5)
ax1.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])
ax2.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])
ax3.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])



plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

df_allsources_g.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="k",alpha=0.1)
ax.set_title("Aperture magnitude (Auxtel - SDSS' filters)")
ax.set_xlabel(" Magnitude (mag)")
ax.set_ylabel(" Magnitude error (mmag)")

# generate legend
import matplotlib.patches as mpatches
green_patch = mpatches.Patch(color='green', label='g band',alpha=0.5)
red_patch = mpatches.Patch(color='red', label='r band',alpha=0.5)
orange_patch = mpatches.Patch(color='orange', label='i band',alpha=0.5)
grey_patch = mpatches.Patch(color='grey', label='z band',alpha=0.5)
black_patch = mpatches.Patch(color='k', label='y band',alpha=0.5)
ax.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])